# Hyperparameter Tuning using HyperDrive

Import all the dependencies that you will need to complete the project.

In [1]:
#Import libraries
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

### Initiate Workspace 

In [2]:
ws = Workspace.from_config()

## Dataset

The dataset is the same that we have used in the AutoML Notebook and for this notebook it will imported directly from the train.py file

### Create Experiment:

In [3]:
#Creating the Experiment
experiment_name = 'hyper-1'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

Create the HyperDrive Configuration needed:

In [4]:
cpu_cluster = "notebook138164"
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({"--C": uniform(0.2, 1),
                             "--max_iter": choice(50, 100, 200)
                             })

#TODO: Create your estimator and hyperdrive config
mydir = os.getcwd()
# Create a SKLearn estimator for use with train.py
est = SKLearn(mydir,compute_target=cpu_cluster, entry_script='train.py')


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=6,
                                    max_concurrent_runs=5)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
#TODO: Submit your experiment
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr= experiment.submit(config=hyperdrive_config,show_output=True)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

Get the best model from the hyperdrive experiments:

In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_3ac32aa7-a722-491b-ab30-5cfcea95ddf7_0

 Accuracy: 0.7900763358778626


In [8]:
#Look up for additional best run metrics
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyper-1,HD_3ac32aa7-a722-491b-ab30-5cfcea95ddf7_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### The best model was found when performing the AutoML therefore no further steps will be required here